In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from keras import applications
from keras.models import Sequential, Model, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv2D, MaxPooling2D, LSTM, Reshape, Permute, TimeDistributed, CuDNNLSTM,LeakyReLU
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
train_path = '/home/livelab/Desktop/NewResizedBalanced/Training/' # directory containing subsets of data with labels
valid_path = '/home/livelab/Desktop/NewResized/Validation/'
test_path = '/home/livelab/Desktop/NewResized/Test/'

In [3]:
def findLayerNumByName(model, name):
    layerList=model.layers
    return layerList.index(model.get_layer(layer_name))

In [4]:
EPOCH_NUM = 30 # number of epochs to run
LEARN_RATE = 0.01 # how much the guesses adjust for loss each time to find the minimum
BATCH_SIZE = 64 # how many to process at once (greatest power of 2 that can ft in RAM)

train_datagen = keras.preprocessing.image.ImageDataGenerator(zca_whitening=True, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0, height_shift_range=0.0, brightness_range=None, 
                                                             shear_range=0.0, zoom_range=0.0, channel_shift_range=0.0, fill_mode='nearest', 
                                                             cval=0.0, horizontal_flip=True, vertical_flip=True)
valid_datagen = keras.preprocessing.image.ImageDataGenerator()
test_datagen = keras.preprocessing.image.ImageDataGenerator()

# Use the ImageDataGenerators to load the training data
train_gen = train_datagen.flow_from_directory(directory=train_path,
                                    target_size=(256,256), # size to resize images to
                                    color_mode='rgb', # color mode of the images
                                    batch_size=BATCH_SIZE, # how many images to process at once
                                    class_mode='categorical', # classify into categorical classes
                                    shuffle=True # shuffle order of images
)
valid_gen = valid_datagen.flow_from_directory(directory=valid_path,
                                    target_size=(256,256),
                                    color_mode='rgb',
                                    batch_size=BATCH_SIZE,
                                    class_mode='categorical',
                                    shuffle=True
)
test_gen = test_datagen.flow_from_directory(directory=test_path,
                                    target_size=(256,256),
                                    color_mode='rgb',
                                    batch_size=1,
                                    class_mode='categorical',
                                    shuffle=False
)

/home/livelab/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:336: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 262258 images belonging to 24 classes.
Found 163247 images belonging to 24 classes.
Found 171954 images belonging to 24 classes.


In [5]:
#full inception
#freeze mix3 and prior
inputData=Input(shape=(256, 256, 3))
model = applications.inception_v3.InceptionV3(weights='imagenet', include_top=False,input_shape = ( 256, 256, 3))
# layer_name = 'mixed6'
# index=findLayerNumByName(model, layer_name)
# intermediate_model=Model(inputs=model.input,
#                                  outputs=model.get_layer(layer_name).output)
# for layer in intermediate_model.layers:
#     layer.trainable = False
intermediateOut = Flatten()(model.output)
intermediateOut = Dense(32)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)
intermediateOut = Dropout(0.4)(intermediateOut)
prediction = Dense(24, activation='softmax')(intermediateOut)
optmr = Adam(lr=0.0005, decay = 5e-4)
model_final = Model(input = model.input, output = prediction)
model_final.compile(loss = "categorical_crossentropy", optimizer = optmr, metrics=["accuracy"])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/livelab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [6]:
model_final.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 127, 127, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 127, 127, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 127, 127, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [7]:
filepath = "model-dev-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto', period=1)
callbacks_list = [checkpoint]

In [8]:
STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size/20
STEP_SIZE_VALID=valid_gen.n//valid_gen.batch_size/10

In [9]:
with tf.device('/GPU:0'):
    model_final.fit_generator(generator=train_gen,
                        steps_per_epoch=STEP_SIZE_TRAIN, # number of steps in each epoch
                        validation_data=valid_gen,
                        validation_steps=STEP_SIZE_VALID,
                        callbacks=callbacks_list,
                        epochs=EPOCH_NUM
    )

Instructions for updating:
Use tf.cast instead.
Epoch 1/30


/home/livelab/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/livelab/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:735: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


205/204 [==============================] - 107s 523ms/step - loss: 2.0403 - acc: 0.4419 - val_loss: 4.3138 - val_acc: 0.0308

Epoch 00001: val_acc improved from -inf to 0.03076, saving model to saved-model-01-0.03.hdf5
Epoch 2/30
205/204 [==============================] - 99s 481ms/step - loss: 1.2220 - acc: 0.6888 - val_loss: 2.7704 - val_acc: 0.2059

Epoch 00002: val_acc improved from 0.03076 to 0.20594, saving model to saved-model-02-0.21.hdf5
Epoch 3/30
205/204 [==============================] - 100s 490ms/step - loss: 0.9165 - acc: 0.7607 - val_loss: 3.0887 - val_acc: 0.1665

Epoch 00003: val_acc did not improve from 0.20594
Epoch 4/30
205/204 [==============================] - 101s 491ms/step - loss: 0.7494 - acc: 0.7960 - val_loss: 2.7481 - val_acc: 0.2138

Epoch 00004: val_acc improved from 0.20594 to 0.21385, saving model to saved-model-04-0.21.hdf5
Epoch 5/30
205/204 [==============================] - 102s 496ms/step - loss: 0.6408 - acc: 0.8179 - val_loss: 2.6159 - val_acc: 

In [10]:
model_final.load_weights("saved-model-26-0.38.hdf5")

In [12]:
TEST_NUM=171954
# evaluate the function using the test set
test_loss, test_acc = model_final.evaluate_generator(test_gen,
                                            TEST_NUM)

# print test accuracy
print('Accuracy: {}'.format(test_acc))

Accuracy: 0.5098282098700815
